In [1]:
import numpy as np

# Euclidean distance

## New axis for broadcasting
In order to vectorize the distance calculation, we need to expand the arrays. This enables broadcasting

In [2]:
centroids = np.array([[0.0, 0.0], [1.0, 1.0], [2.0, 2.0]])
centroids.shape

(3, 2)

In [3]:
centroids_expanded = centroids[np.newaxis, :, :]
centroids_expanded.shape

(1, 3, 2)

In [4]:
points = np.array([[0.0, 0.0], [1.0, 1.0], [2.0, 2.0], [3.0, 3.0]])
points.shape

(4, 2)

In [5]:
points_expanded = points[:, np.newaxis, :]
points_expanded.shape

(4, 1, 2)

In [6]:
dist = points_expanded - centroids_expanded
dist.shape

(4, 3, 2)

In [7]:
dist

array([[[ 0.,  0.],
        [-1., -1.],
        [-2., -2.]],

       [[ 1.,  1.],
        [ 0.,  0.],
        [-1., -1.]],

       [[ 2.,  2.],
        [ 1.,  1.],
        [ 0.,  0.]],

       [[ 3.,  3.],
        [ 2.,  2.],
        [ 1.,  1.]]])

## Sum of squared distances and Euc. distance

In [8]:
sum_squared = np.sum(dist**2, axis=2)
sum_squared

array([[ 0.,  2.,  8.],
       [ 2.,  0.,  2.],
       [ 8.,  2.,  0.],
       [18.,  8.,  2.]])

In [9]:
euc_dist = np.sqrt(sum_squared)
euc_dist

array([[0.        , 1.41421356, 2.82842712],
       [1.41421356, 0.        , 1.41421356],
       [2.82842712, 1.41421356, 0.        ],
       [4.24264069, 2.82842712, 1.41421356]])

## Function

In [10]:
def euclidean_distance(points: np.array, centroids: np.array) -> np.array:
    # 1. Create views on both arrays with an additional axisto use broadcasting
    points_expanded = points[:, np.newaxis, :]
    centroids_expanded = centroids[np.newaxis, :, :]
    
    # 2. Calculate the sum of squared differences for each point-centroid-combination
    # The sum is over axis=2, because axis=0 contains all points, axis=1 contains all centroids and axis=2 contains the respective coordinates 
    sum_squared = np.sum((points_expanded - centroids_expanded)**2, axis=2)
    
    # 3. Return the square root of the sum of squared distances to get euclidean distance
    return np.sqrt(sum_squared)

# Update centroids

## Select centroid for each point based on minimal distance 

In [11]:
selected_centroids = np.argmin(euc_dist, axis=1)
selected_centroids

array([0, 1, 2, 2])

## Use selected_centroids for subsetting
The selected_centroids array can be used as a mask array to get only the points assigned to that specific centroid

In [12]:
cluster2 = points[selected_centroids==2]
cluster2

array([[2., 2.],
       [3., 3.]])

In [13]:
np.mean(cluster2, axis=0)

array([2.5, 2.5])

## Update the centroids in a loop

In [14]:
for i in range(centroids.shape[0]):
    cluster = points[selected_centroids==i]
    new_centroid = np.mean(cluster, axis=0)
    centroids[i, :] = new_centroid

In [15]:
centroids

array([[0. , 0. ],
       [1. , 1. ],
       [2.5, 2.5]])

## Function

In [16]:
def update_centroids(points: np.array, centroids: np.array, euc_dist: np.array) -> np.array:
    selected_centroids = np.argmin(euc_dist, axis=1)
    for i in range(centroids.shape[0]):
        cluster = points[selected_centroids==i]
        new_centroid = np.mean(cluster, axis=0)
        centroids[i, :] = new_centroid
    
    return centroids

# Validate
Below, I will validate the code using the example points from class

## Initialize points and centroids

In [17]:
rng = np.random.default_rng(seed = 1234)
cl1 = rng.multivariate_normal([-2,-2], [[1,-0.5],[-0.5,1]], size=100)
cl2 = rng.multivariate_normal([1,0], [[1,0],[0,1]], size=150)
cl3 = rng.multivariate_normal([3,2], [[1,-0.7],[-0.7,1]], size=200)
pts = np.concatenate((cl1,cl2,cl3))

Select three random centroids

In [18]:
idx = np.random.randint(pts.shape[0], size=3)
centroids = pts[idx]
centroids

array([[ 0.52484033,  0.02424452],
       [ 1.73118867, -0.22964706],
       [-1.7258032 , -0.83710488]])

## Iterate until centroids do not change anymore

In [22]:
euc_dist = euclidean_distance(pts, centroids)

In [23]:
centroids = update_centroids(pts, centroids, euc_dist)

In [24]:
centroids

array([[ 0.81048433,  0.37567953],
       [ 2.99160512,  1.72573031],
       [-1.86160111, -1.83338785]])

In [25]:
converged = False

In [26]:
while not converged:
    old_centroids = np.copy(centroids)
    euc_dist = euclidean_distance(pts, centroids)
    centroids = update_centroids(pts, centroids, euc_dist)
    if np.array_equal(old_centroids, centroids):
        converged = True

In [27]:
centroids

array([[ 0.90582982, -0.16728387],
       [ 2.87614469,  1.95184761],
       [-2.03956666, -1.85662027]])